<a href="https://colab.research.google.com/github/IvanDePivan/2AMM30-groep-2-component-1/blob/main/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook contains an old model (Named Entity Recognition with Spacy) that is not used as final model. We did not adapt this model to be able to run from anyones personal computer. Therefore, this notbook will likely not run, but it still provides a general idea of what we did.

In [2]:
import pandas as pd
import json
import numpy as np
import spacy
from spacy.training.example import Example #holds information for one training instance
import random
from tqdm import tqdm
from spacy.scorer import Scorer
import warnings
warnings.filterwarnings('ignore')
import os
import glob
import re

In [5]:
# Authenticate
from google.colab import drive
drive.mount('/content/drive')
from google.colab import auth
auth.authenticate_user()

# Get email of current Colab user
import requests
gcloud_token = !gcloud auth print-access-token
gcloud_tokeninfo = requests.get('https://www.googleapis.com/oauth2/v3/tokeninfo?access_token=' + gcloud_token[0]).json()
email = gcloud_tokeninfo['email']

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
# Define filepath
if email == '':
  filename = "/content/drive/MyDrive/TUe/TM/Exports/export_41675_project-41675-at-2023-10-04-09-37-9bbbec63.json"
elif email == '':
  filename = "/content/drive/MyDrive/Text Mining/export_41675_project-41675-at-2023-10-04-12-08-05f5e3f5.json"

In [7]:
DATA = []

# read json file
with open(filename, 'rb') as fp:
  training_data = json.load(fp)

  # get text, labels, relations, benchmark for each article
  for article in training_data:
    entities = []
    id_entities = []
    relations = []
    original_text = article.get('data').get('text')
    if len(article.get('annotations')) == 5: # N=5
      benchmark = True
    else:
      benchmark = False
    for annotation in article.get('annotations'):
      if annotation.get('ground_truth'): # only include ground truth
        for ind, label in enumerate(annotation.get('result')):
          if label.get('type') == 'labels':
            start = label.get('value').get('start')
            end = label.get('value').get('end')
            id = label.get('id')
            text = label.get('value').get('text')
            label = label.get('value').get('labels')[0] # note: cannot deal with multiple labels
            id_entities.append((start, end, label, id, text)) # WHY ORDER WEIRD FOR TRAIN[0]?
            entities.append((start, end, label))
          elif label.get('type') == 'relation':
            from_id = label.get('from_id')
            to_id = label.get('to_id')
            relation = label.get('labels')[0] # note: cannot deal with multiple relations
            relations.append((from_id, to_id, relation))
          else:
            print("found unknown label type (no label or relation)")

    # append article to training data as dictionary
    DATA.append({
    'text': original_text,
    'entities': entities,
    'id_entities': id_entities,
    'relations': relations,
    'benchmark': benchmark
    })

In [8]:
# Remove entities with smallest span in case of overlapping entities
def remove_overlap(entities):
  # Initialize a list to store the final non-overlapping entities
  final_entities = []

  # Sort the entities by their start position in ascending order
  entities.sort(key=lambda entity: entity[0])

  # Iterate through the sorted entities
  for entity in entities:
      overlaps = False
      for existing_entity in final_entities:
          # Check for overlapping entities
          if (existing_entity[0] <= entity[0] < existing_entity[1]) or \
            (existing_entity[0] < entity[1] <= existing_entity[1]):
              overlaps = True
              break
      if not overlaps:
          final_entities.append(entity)

  return final_entities

for article in DATA:
  article['entities'] = remove_overlap(article['entities'])

In [9]:
def train_test_split(data,test_size):
  train_end= int(len(data)*(1-test_size))
  return data[0:train_end], data[train_end:len(data)]

In [10]:
train, test =  train_test_split(DATA,0.25)

In [19]:
from spacy.tokens import DocBin

nlp = spacy.blank('en')
db = DocBin()

for article in tqdm(train):
  doc = nlp.make_doc(article['text'])
  ents = []
  for start, end, label in article['entities']:
    span = doc.char_span(start, end, label=label, alignment_mode='contract')
    if span is None:
      pass
    else:
      ents.append(span)
    doc.ents = ents
    db.add(doc)

db.to_disk('./train.spacy')

100%|██████████| 30/30 [00:00<00:00, 36.52it/s]


In [20]:
nlp = spacy.blank('en')
db = DocBin()

for article in tqdm(test):
  doc = nlp.make_doc(article['text'])
  ents = []
  for start, end, label in article['entities']:
    span = doc.char_span(start, end, label=label, alignment_mode='contract')
    if span is None:
      pass
    else:
      ents.append(span)
    doc.ents = ents
    db.add(doc)

db.to_disk('./test.spacy')

100%|██████████| 10/10 [00:00<00:00, 60.15it/s]


In [26]:
!cp "/content/drive/MyDrive/TUe/TM/base_config.cfg" ./

In [32]:
!python -m spacy init fill-config base_config.cfg config.cfg

2023-11-08 14:33:52.580171: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-08 14:33:52.580243: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-08 14:33:52.580309: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-08 14:33:54.385267: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [33]:
!python -m spacy train config.cfg --output "/content/drive/MyDrive/TUe/TM/output" --paths.train ./train.spacy --paths.dev ./test.spacy --gpu-id 0

2023-11-08 14:34:37.266226: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-08 14:34:37.266286: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-08 14:34:37.266322: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-08 14:34:38.416630: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
ℹ Saving to output directory: /content/drive/MyDrive/TUe/TM/output
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
Some weights of RobertaModel were not initialized from the model checkpoint a

In [3]:
best_nlp =  spacy.load('/content/drive/MyDrive/TUe/TM/output/model-best')

In [25]:
doc = best_nlp(test[1]['text'])
colors = {'Winner': '#F67DE3', 'Date':'#7DF6D9', 'Prizetype': '#DC143C', 'Reason': '#00FF00', 'Nationality': '#E287433' }
options = {'colors':colors}
spacy.displacy.render(doc, style='ent', options=options, jupyter=True)